In [2]:
import pandas as pd
from datetime import datetime

In [68]:
power = pd.read_csv('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Power/Power_Data_Full_ChillerValuesOnly.csv')

In [69]:
tonnage = pd.read_csv('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/Load_Data_Full_ChillerValuesOnly.csv')

In [70]:
kw_ton = pd.read_csv('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/kw_per_ton_data_02122019_02122020.csv')

In [71]:
# add dates to kw_ton dataframe
dates = list(tonnage['Date/Time'])
kw_ton.insert(0,'Date/Time',dates)

In [72]:
# compile total values for all chillers
totalkw = []
totaltons = []
totalkw_ton = []
for i in range(len(kw_ton)):
    totalkw.append(sum(power.iloc[i,1:]))
    totaltons.append(sum(tonnage.iloc[i,1:]))
    totalkw_ton.append(sum(kw_ton.iloc[i,1:]))

In [73]:
totals = pd.DataFrame({'kW':totalkw,'Tons':totaltons,'kW/Ton':totalkw_ton,'Diff':[abs(x/y - z) for x,y,z in zip(totalkw,totaltons,totalkw_ton)]})

In [74]:
# All rates given in $/kWh
son = .045568 #summer on peak rate (May-Sept 2pm-8pm M-F)
soff = 0.023985 # summer off peak rate (May-Sept all times not On Peak)
won = 0.029581 # winter on peak rate (Oct-Apr 6am-10am and 5pm-9pm M-F)
woff = 0.024352 # winter off peak rate (Oct-Apr all times not On Peak)

In [101]:
date = dates[0]
int(date.split(' ')[0].split('-')[1]) >= 5 and int(date.split(' ')[0].split('-')[1]) <= 9

False

In [75]:
prices = []
for date in dates:
    if (int(date.split(' ')[0].split('-')[1]) >= 5 and int(date.split(' ')[0].split('-')[1]) <= 9):
        if (int(date.split(' ')[1].split(':')[0]) >= 14 and int(date.split(' ')[1].split(':')[0]) <= 19):
            prices.append(son)
        else:
            prices.append(soff)
    else:
        if ((int(date.split(' ')[1].split(':')[0]) >= 6 and int(date.split(' ')[1].split(':')[0])<=9) or (int(date.split(' ')[1].split(':')[0])>=17 and int(date.split(' ')[1].split(':')[0])<=20)):
            prices.append(won)
        else:
            prices.append(woff)
prices_df = pd.DataFrame({'Date/Time':dates,'Price':prices})

In [13]:
power.head()

,Date/Time,AHSC_CHL01_ChlrPwr,AHSC_CHL02_ChlrPwr,AHSC_CHL03_ChlrPwr,AHSC_CHL04_ChlrPwr,AHSC_CHL07_ChlrPwr,AHSC_CHL08_ChlrPwr,CRB_CHL05_ChlrPwr,CRB_CHL06_ChlrPwr,CRB_CHL07_ChlrPwr,CRB_CHL08_ChlrPwr,CRB_CHL09_ChlrPwr,CRB_CHL10_ChlrPwr,CHRP_CHL06_ChlrPwr,CHRP_CHL07_ChlrPwr,CHRP_CHL08_ChlrPwr,CHRP_CHL09_ChlrPwr,CHRP_CHL11_ChlrPwr,CHRP_CHL12_ChlrPwr
0,2019-02-12 00:00:00,0.0,0.0,0.0,556.2,0.0,0.0,0.0,0.0,430.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1020.9,0.0
1,2019-02-12 00:02:00,0.0,0.0,0.0,566.1,0.0,0.0,0.0,0.0,430.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1020.9,0.0
2,2019-02-12 00:04:00,0.0,0.0,0.0,572.0,0.0,0.0,0.0,0.0,427.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1018.3,0.0
3,2019-02-12 00:06:00,0.0,0.0,0.0,577.9,0.0,0.0,0.0,0.0,427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1012.8,0.0
4,2019-02-12 00:08:00,0.0,0.0,0.0,568.8,0.0,0.0,0.0,0.0,426.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1007.4,0.0


In [14]:
avgCRB7 = power['CRB_CHL07_ChlrPwr'].mean()
totalkw = [x + avgCRB7 * (1 + 1.3 + 1.2) for x in totalkw]

In [76]:
prices_df.insert(2,'kW',totalkw)

In [80]:
prices_df.head()

,Date/Time,Price,kW,New Cost,Cost
0,2019-02-12 00:00:00,0.024352,2007.7,1.629717,48.891510
1,2019-02-12 00:02:00,0.024352,2017.1,1.637347,49.120419
2,2019-02-12 00:04:00,0.024352,2017.9,1.637997,49.139901
3,2019-02-12 00:06:00,0.024352,2017.7,1.637834,49.135030
4,2019-02-12 00:08:00,0.024352,2002.7,1.625658,48.769750


In [49]:
total_cost = sum(prices_df['Cost'])
round(total_cost,2)

1378121.39

In [54]:
new_power = pd.read_csv('/Users/ethanweiss/Downloads/kW_2019to2020_Upd.csv',index_col = 0)

In [45]:
dates = list(new_power['Time'])
prices = []
for date in dates:
    if (int(date.split(' ')[0].split('-')[1]) >= 5 and int(date.split(' ')[0].split('-')[1]) <= 9):
        if (int(date.split(' ')[1].split(':')[0]) >= 14 and int(date.split(' ')[1].split(':')[0]) <= 19):
            prices.append(son)
        else:
            prices.append(soff)
    else:
        if ((int(date.split(' ')[1].split(':')[0]) >= 6 and int(date.split(' ')[1].split(':')[0])<=9) or (int(date.split(' ')[1].split(':')[0])>=17 and int(date.split(' ')[1].split(':')[0])<=20)):
            prices.append(won)
        else:
            prices.append(woff)
new_power.insert(3,'Price',prices)

In [ ]:
cost = []
for i in range(len(prices_df)):
    cost.append(1/30 * prices_df.loc[i,'Price'] * prices_df.loc[i,'kW'])
prices_df.insert(4,'Cost',cost)

In [48]:
total_cost = sum(new_power['Cost'])
round(total_cost,2)

680976.25

In [59]:
correct = new_power[new_power['Is CL Correct'] == 1]

In [61]:
correct['kW'].mean()

2757.657598172153

In [63]:
dates = list(correct['Time'])
prices = []
for date in dates:
    if (int(date.split(' ')[0].split('-')[1]) >= 5 and int(date.split(' ')[0].split('-')[1]) <= 9):
        if (int(date.split(' ')[1].split(':')[0]) >= 14 and int(date.split(' ')[1].split(':')[0]) <= 19):
            prices.append(son)
        else:
            prices.append(soff)
    else:
        if ((int(date.split(' ')[1].split(':')[0]) >= 6 and int(date.split(' ')[1].split(':')[0])<=9) or (int(date.split(' ')[1].split(':')[0])>=17 and int(date.split(' ')[1].split(':')[0])<=20)):
            prices.append(won)
        else:
            prices.append(woff)
correct.insert(5,'Price',prices)

In [103]:
correct.head()

,Time,Average Tons,kW,Old kw,Calculated CL,Is CL Correct,Price,Cost,Old cost
49,2019-02-14 00:59:59.755,6990.274424,3001.086623,2894.1,6990.274424,1.0,0.024352,73.082461,70.477123
57,2019-02-14 08:59:59.715,7189.172845,3081.380043,2763.8,7189.172845,1.0,0.029581,91.150303,81.755968
98,2019-02-16 01:59:59.510,7218.839035,2911.261532,2840.5,7218.839035,1.0,0.024352,70.895041,69.171856
149,2019-02-18 05:00:00.240,4828.621940,1705.543827,1782.7,4828.621940,1.0,0.024352,41.533403,43.412310
150,2019-02-18 06:00:00.245,4872.548053,1742.415487,1836.3,4872.548053,1.0,0.029581,51.542393,54.319590


In [119]:
(sum(correct['Old kw']) - sum(correct['kW'])) #* (365 * 24)/909

92906.94326151535

In [101]:
old_kw = []
cost = []
old_cost = []
for i in range(len(correct)):
    new_kwVal = prices_df.loc[correct.index[i] * 30,'kW']
    old_kw.append(new_kwVal)
    cost.append(correct.loc[correct.index[i],'Price'] * correct.loc[correct.index[i],'kW'])
    old_cost.append(new_kwVal * prices_df.loc[correct.index[i] * 30,'Price'])

In [102]:
correct.insert(3,'Old kw',old_kw)
correct.insert(7,'Cost',cost)
correct.insert(8,'Old cost',old_cost)

In [104]:
old_total = round(sum(correct['Old cost']),2)
total = round(sum(correct['Cost']),2)
print(old_total)
print(total)

67294.06
64963.74


In [114]:
avg_hourly_saved = (old_total - total)

In [115]:
avg_hourly_saved

2330.3199999999997

In [113]:
len(correct)

909

In [116]:
avg_hourly_saved/len(correct)

2.563608360836083

In [109]:
avg_hourly_saved * 365 * 24

22457.209240924087

In [90]:
prices_df.head()

,Date/Time,Price,kW,New Cost,Cost
0,2019-02-12 00:00:00,0.024352,2007.7,1.629717,48.891510
1,2019-02-12 00:02:00,0.024352,2017.1,1.637347,49.120419
2,2019-02-12 00:04:00,0.024352,2017.9,1.637997,49.139901
3,2019-02-12 00:06:00,0.024352,2017.7,1.637834,49.135030
4,2019-02-12 00:08:00,0.024352,2002.7,1.625658,48.769750


In [ ]:
old_price = []
for date in dates:
    for i in range(len(new_power))

In [17]:
kw_ton.head()

,Date/Time,AHSC_CHL01_kwPerTon,AHSC_CHL02_kwPerTon,AHSC_CHL03_kwPerTon,AHSC_CHL04_kwPerTon,AHSC_CHL07_kwPerTon,AHSC_CHL08_kwPerTon,CRB_CHL05_kwPerTon,CRB_CHL06_kwPerTon,CRB_CHL07_kwPerTon,CRB_CHL08_kwPerTon,CRB_CHL09_kwPerTon,CRB_CHL10_kwPerTon,CHRP_CHL06_kwPerTon,CHRP_CHL07_kwPerTon,CHRP_CHL08_kwPerTon,CHRP_CHL09_kwPerTon,CHRP_CHL11_kwPerTon,CHRP_CHL12_kwPerTon
0,2019-02-12 00:00:00,0.0,0.0,0.0,0.530876,0.0,0.0,0.0,0.0,0.482135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.468046,0.0
1,2019-02-12 00:02:00,0.0,0.0,0.0,0.535343,0.0,0.0,0.0,0.0,0.486572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.469027,0.0
2,2019-02-12 00:04:00,0.0,0.0,0.0,0.540240,0.0,0.0,0.0,0.0,0.482530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.470967,0.0
3,2019-02-12 00:06:00,0.0,0.0,0.0,0.545116,0.0,0.0,0.0,0.0,0.484160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.468226,0.0
4,2019-02-12 00:08:00,0.0,0.0,0.0,0.535423,0.0,0.0,0.0,0.0,0.485920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.465729,0.0


In [6]:
tonnage.head()

,Date/Time,AHSC_CHL01_TONS,AHSC_CHL02_TONS,AHSC_CHL03_TONS,CHRP_CHL08_TONS,CHRP_CHL09_TONS,CHRP_CHL06_TONS,CHRP_CHL07_TONS,CHRP_CHL10_TONS,CHRP_CHL11_TONS,...,CRB_CHL07_TONS,CRB_CHL08_TONS,CRB_CHL09_TONS,CRB_CHL10_TONS,CRB_CHL01B_TONS,AHSC_CHL04_TONS,AHSC_CHL07_TONS,AHSC_CHL08_TONS,CRB_CHL02B_TONS,CRB_CHL03B_TONS
0,2019-02-12 00:00:00,1208.309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2181.195,...,893.11150,0.0,0.0,0.0,0.0,1047.702,0.0,0.0,0.0,0.0
1,2019-02-12 00:02:00,1211.706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2176.634,...,883.93870,0.0,0.0,0.0,0.0,1057.453,0.0,0.0,0.0,0.0
2,2019-02-12 00:04:00,1212.154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2162.145,...,886.16180,0.0,0.0,0.0,0.0,1058.788,0.0,0.0,0.0,0.0
3,2019-02-12 00:06:00,1208.656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2163.059,...,881.93895,0.0,0.0,0.0,0.0,1060.142,0.0,0.0,0.0,0.0
4,2019-02-12 00:08:00,1211.696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2163.059,...,877.71610,0.0,0.0,0.0,0.0,1062.337,0.0,0.0,0.0,0.0


In [8]:
power.head()

,Date/Time,AHSC_CHL01_ChlrPwr,AHSC_CHL02_ChlrPwr,AHSC_CHL03_ChlrPwr,AHSC_CHL04_ChlrPwr,AHSC_CHL07_ChlrPwr,AHSC_CHL08_ChlrPwr,CRB_CHL05_ChlrPwr,CRB_CHL06_ChlrPwr,CRB_CHL07_ChlrPwr,CRB_CHL08_ChlrPwr,CRB_CHL09_ChlrPwr,CRB_CHL10_ChlrPwr,CHRP_CHL06_ChlrPwr,CHRP_CHL07_ChlrPwr,CHRP_CHL08_ChlrPwr,CHRP_CHL09_ChlrPwr,CHRP_CHL11_ChlrPwr,CHRP_CHL12_ChlrPwr
0,2019-02-12 00:00:00,0.0,0.0,0.0,556.2,0.0,0.0,0.0,0.0,430.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1020.9,0.0
1,2019-02-12 00:02:00,0.0,0.0,0.0,566.1,0.0,0.0,0.0,0.0,430.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1020.9,0.0
2,2019-02-12 00:04:00,0.0,0.0,0.0,572.0,0.0,0.0,0.0,0.0,427.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1018.3,0.0
3,2019-02-12 00:06:00,0.0,0.0,0.0,577.9,0.0,0.0,0.0,0.0,427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1012.8,0.0
4,2019-02-12 00:08:00,0.0,0.0,0.0,568.8,0.0,0.0,0.0,0.0,426.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1007.4,0.0


In [5]:
sum(power['AHSC_CHL01_ChlrPwr'])

0.0